# Create Training Dataset

In [3]:
# load metadata
import json
with open('../../data/metadata_postgres_with_licenses_deduplicated.json', 'r') as f:
    metadata = json.load(f)

In [70]:
# create training data
import pandas as pd
only_open_licenses = False
open_licenses = ["mit", "apache-2.0"]
columns = ["dataset", "table_name_a", "table_name_b", "columns_a", "columns_b", "primary_keys_a", "primary_keys_b", "key_a", "key_b"]
rows = []
for dataset in metadata.keys():
    if only_open_licenses and metadata[dataset]["INFO"]["license"] not in open_licenses:
        continue
    for table_name_a in metadata[dataset]["TABLES"].keys():
        columns_a = [col[0] for col in metadata[dataset]["TABLES"][table_name_a]["COLUMNS"]]
        for fks in metadata[dataset]["TABLES"][table_name_a]["FOREIGN_KEYS"]:
            keys_a = fks["FOREIGN_KEY"]
            keys_b = fks["REFERENCE_COLUMN"]
            # skip composite keys
            if len(keys_a) != 1 or len(keys_b) != 1:
                continue
            key_a = keys_a[0]
            key_b = keys_b[0]
            table_name_b = fks["REFERENCE_TABLE"]
            primary_keys_a = metadata[dataset]["TABLES"][table_name_a]["PRIMARY_KEYS"]
            columns_b = []
            primary_keys_b = []
            if table_name_b in metadata[dataset]["TABLES"]:
                columns_b = [col[0] for col in metadata[dataset]["TABLES"][table_name_b]["COLUMNS"]]
                primary_keys_b = metadata[dataset]["TABLES"][table_name_b]["PRIMARY_KEYS"]
                # skip composite keys
                if len(primary_keys_a) > 1 or len(primary_keys_b) > 1:
                    continue

            primary_key_a = primary_keys_a[0] if len(primary_keys_a) != 0 else ""
            primary_key_b = primary_keys_b[0] if len(primary_keys_b) != 0 else ""
            row = [dataset, table_name_a, table_name_b, columns_a, columns_b, primary_key_a, primary_key_b, key_a, key_b]
            rows.append(row)
df = pd.DataFrame(rows, columns=columns)

In [71]:
# save training data
df.to_csv("../../data/fk_detection_dataset_deduplicated.csv",index=False)
df.to_parquet("../../data/fk_detection_dataset_deduplicated.parquet")

# Evaluation

In [72]:
df

,dataset,table_name_a,table_name_b,columns_a,columns_b,primary_keys_a,primary_keys_b,key_a,key_b
0,000000_tables.sql,address,state,"[id, uuid, flat_buil_number, locality, city, p...","[id, uuid, state_name]",id,id,state_id,id
1,000000_tables.sql,customer_address,customer,"[id, customer_id, address_id]","[id, uuid, firstname, lastname, email, contact...",id,id,customer_id,id
2,000000_tables.sql,customer_address,address,"[id, customer_id, address_id]","[id, uuid, flat_buil_number, locality, city, p...",id,id,address_id,id
3,000000_tables.sql,brands,address,"[id, uuid, brand_name, customer_rating, number...","[id, uuid, flat_buil_number, locality, city, p...",id,id,address_id,id
4,000000_tables.sql,category_item,item,"[id, item_id, category_id]","[id, uuid, item_name, price]",id,id,item_id,id
...,...,...,...,...,...,...,...,...,...
105230,695776_V1.0__Initial_Schema.sql,hr_attendance,res_users,"[id, create_uid, create_date, write_date, writ...","[id, active, login, password, company_id, part...",id,id,write_uid,id
105231,695776_V1.0__Initial_Schema.sql,hr_timesheet_sheet_sheet,res_users,"[id, create_uid, create_date, write_date, writ...","[id, active, login, password, company_id, part...",id,id,create_uid,id
105232,695776_V1.0__Initial_Schema.sql,hr_timesheet_sheet_sheet,hr_department,"[id, create_uid, create_date, write_date, writ...","[id, create_uid, create_date, write_date, writ...",id,id,department_id,id
105233,695776_V1.0__Initial_Schema.sql,hr_timesheet_sheet_sheet,hr_employee,"[id, create_uid, create_date, write_date, writ...","[id, create_uid, create_date, write_date, writ...",id,id,employee_id,id


In [84]:
subset1 = df["key_a"]==df["key_b"]
print(f"foreign_key == primary_key: {subset1.sum()}/{df['key_a'].count()}")
df[subset1].head()

foreign_key == primary_key: 40348/105235


,dataset,table_name_a,table_name_b,columns_a,columns_b,primary_keys_a,primary_keys_b,key_a,key_b,distance
12,000009_schema.sql,dept_emp,employees,"[emp_no, dept_no]","[emp_no, emp_title_id, birth_date, first_name,...",,,emp_no,emp_no,0
13,000009_schema.sql,dept_emp,departments,"[emp_no, dept_no]","[dept_no, dept_name]",,,dept_no,dept_no,0
14,000009_schema.sql,dept_manager,departments,"[dept_no, emp_no]","[dept_no, dept_name]",,,dept_no,dept_no,0
15,000009_schema.sql,dept_manager,employees,"[dept_no, emp_no]","[emp_no, emp_title_id, birth_date, first_name,...",,,emp_no,emp_no,0
16,000009_schema.sql,salaries,employees,"[emp_no, salary]","[emp_no, emp_title_id, birth_date, first_name,...",,,emp_no,emp_no,0


In [83]:
subset2 = df["key_a"]==df["table_name_b"]+df["key_b"]
print(f"foreign_key == table_id+primary_key: {subset2.sum()}/{df['key_a'].count()}")
df[subset2].head()

foreign_key == table_id+primary_key: 1905/105235


,dataset,table_name_a,table_name_b,columns_a,columns_b,primary_keys_a,primary_keys_b,key_a,key_b,distance
173,000260_event_room.ddl,event_room,event,"[eventid, roomid, position]",[],eventid,,eventid,id,5
174,000260_event_room.ddl,event_room,room,"[eventid, roomid, position]",[],eventid,,roomid,id,4
325,001075_database.pg.sql,accountroles,account,"[accountid, role]","[id, username, firstname, lastname, password]",,id,accountid,id,7
326,001075_database.pg.sql,remarks,workspace,"[workspaceid, remarks]","[id, owner, title, problem, defaultscenarioid]",workspaceid,id,workspaceid,id,9
327,001075_database.pg.sql,subproblem,workspace,"[id, workspaceid, definition, title]","[id, owner, title, problem, defaultscenarioid]",id,id,workspaceid,id,9


In [75]:
subset3 = df["key_a"]==df["table_name_b"]+"_"+df["key_b"]
print(f"foreign_key == table_id_primary_key: {subset3.sum()}/{df['key_a'].count()}")
df[subset3].head()

foreign_key == table_id_primary_key: 12385/105235


,dataset,table_name_a,table_name_b,columns_a,columns_b,primary_keys_a,primary_keys_b,key_a,key_b
0,000000_tables.sql,address,state,"[id, uuid, flat_buil_number, locality, city, p...","[id, uuid, state_name]",id,id,state_id,id
1,000000_tables.sql,customer_address,customer,"[id, customer_id, address_id]","[id, uuid, firstname, lastname, email, contact...",id,id,customer_id,id
2,000000_tables.sql,customer_address,address,"[id, customer_id, address_id]","[id, uuid, flat_buil_number, locality, city, p...",id,id,address_id,id
3,000000_tables.sql,brands,address,"[id, uuid, brand_name, customer_rating, number...","[id, uuid, flat_buil_number, locality, city, p...",id,id,address_id,id
4,000000_tables.sql,category_item,item,"[id, item_id, category_id]","[id, uuid, item_name, price]",id,id,item_id,id


In [85]:
subsetnot123 = ~(subset1 | subset2 | subset3)
print(f"foreign_key != table_id or primary_key: {subsetnot123.sum()}/{df['key_a'].count()}")
df[subsetnot123].head()

foreign_key != table_id or primary_key: 50597/105235


,dataset,table_name_a,table_name_b,columns_a,columns_b,primary_keys_a,primary_keys_b,key_a,key_b,distance
7,000000_tables.sql,brands_item,brands,"[id, item_id, brand_id]","[id, uuid, brand_name, customer_rating, number...",id,id,brand_id,id,6
9,000000_tables.sql,brands_category,brands,"[id, brand_id, category_id]","[id, uuid, brand_name, customer_rating, number...",id,id,brand_id,id,6
11,000009_schema.sql,employees,titles,"[emp_no, emp_title_id, birth_date, first_name,...","[title_id, title]",,,emp_title_id,title_id,4
17,000036_12.sql,dashboard,users,"[id, name, description, scope, groups, owner_i...",[],id,,owner_id,id,6
18,000044_V21__rolle_recht__create.sql,rolle_recht,rolle,"[rolle_recht_rolle_id, rolle_recht_recht_id]",[],,,rolle_recht_rolle_id,rolle_id,12
